In [ ]:
from azureml.core import Workspace, Experiment

#workspace and resources management
ws = Workspace.get(name="quick-starts-ws-129633")#due to resource group management policy we are not able to create a new one, so using the available one
exp = Experiment(workspace=ws, name="udacity-iru-project") 

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


#name of the compute cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already (good practice extracted by Microsoft official documentation: https://docs.microsoft.com/)
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
#creating compute (cpu) cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy,MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import GridParameterSampling
from azureml.train.hyperdrive import choice
import os


# Specify parameter sampler

#define parameter search space -->random sampling over a hyperparameter search space (this would be good for first approach)
# dictionary{key:parameter, value: distribution}

ps = RandomParameterSampling( {
        "--C": uniform(0.1, 1),
        "--max_iter": choice(25, 50, 75, 100)})



# Specify a Policy
# early termination for bad performance runs
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)


# Create a SKLearn estimator for use with train.py

est = SKLearn(source_directory='.',#folder for experiment configuration files.
              entry_script=  'train.py', #relative path to the file used to start training (training script)
              compute_target=cpu_cluster)
                     
primary_metric_name='Accuracy'
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name=primary_metric_name,
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=100)
                                #max_total_runs=40)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

#submiting the exp
hyperdrive_run = exp.submit(hyperdrive_config)
#visualizing `progress of runs
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

#finding the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\nAccuracy of best run :', best_run_metrics['Accuracy'])
print('\nHyperparam values of best run:',parameter_values['arguments'])


#saving the best model--> find the best one 
model = best_run.register_model(model_name='best_model_lr', model_path='outputs/model.joblib')

Best Run Id:  HD_ff3bd351-ca99-435b-b314-e69513198f08_86

Accuracy of best run : 0.9074355083459787

Hyperparam values of best run: ['--C', '0.8228883106324361', '--max_iter', '75']


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"


#creating a tabularDataset
ds =TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [7]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#concatenate features and target
data = pd.concat([x, y], axis=1)
#creating dir 'data'
if not os.path.isdir('data'):
    os.mkdir('data')



In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data, #note: this should contain both features and label/target
    label_column_name='y',
    debug_log='automated_ml_errors.log',
    featurization= 'auto',
    #verbosity= logging.INFO)
    n_cross_validations=5)


In [9]:
# Submit your automl run

automl_run = exp.submit(automl_config,show_output=True)

Running on local machine
Parent Run ID: AutoML_6bd9e1ea-c977-4878-bdea-c833f66b4cba

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [10]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print('Best run:',best_run)
print('Best fitted model:', fitted_model)
model = best_run.register_model(model_name='best_automodel_lr_automl.pkl', model_path='./outputs/')

Best run: Run(Experiment: udacity-iru-project,
Id: AutoML_6bd9e1ea-c977-4878-bdea-c833f66b4cba_53,
Type: None,
Status: Completed)
Best fitted model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                    

In [ ]:
#adding a cell for deleting the compute cluster
cpu_cluster.delete()